## Avaliação - Classificação Automática de Segmentos de Imagens 

Nesta prática você irá avaliar um dataset de 1.500 segmentos de imagens. Nesse projeto, cada instancia representa um segmento de 3x3 pixels de uma imagem de algum dos seguintes elementos:

<img src="segments.png" alt="Imagens que foram seguementadas">

Assim, esta tarefa consiste em classificar tais segmentos de 3x3 pixels em um dos tipos de imagens externas (cimento, janela, grama, etc.). Cada instancia é representada da seguinte forma: 

<ol>
    <li>region-centroid-col:  média do valor dos pixels (coluna) </li>
    <li>region-centroid-row:  média do valor dos pixels (linha) </li>
    <li>region-pixel-count:  o número de pixels em uma região(3x3 = 9 neste caso) </li>
    <li>short-line-density-5: resultados de uma linha extraída no algoritmo que conta quantas linhas de comprimento 5 (qualquer orientação) com baixo contraste, menor ou igual a 5, passam pela região. </li>
    <li>short-line-density-2:  igual a densidade de linha curta-5, mas conta linhas de alto contraste, maiores que 2 </li>
    <li>vedge-mean: mede o contraste de pixels adjacentes horizontalmente na região. Existem 6, a média e o desvio padrão são dados. Este atributo é usado como um detector de borda vertical.</li>
    <li>vegde-sd: desvio padrão do contraste de pixels adjacentes horizontalmente </li>
    <li>hedge-mean: mede o contraste de pixels adjacentes verticalmente. Usado para detecção de linha horizontal. </li>
    <li>hedge-sd: desvio padrão do contraste de pixels adjacentes verticalmente.</li>
    <li>intensity-mean:  a média na região de (R + G + B) / 3 </li>
    <li>rawred-mean: a média sobre a região do valor R (cor vermelha) </li>
    <li>rawblue-mean: a média sobre a região do valor B (cor azul) </li>
    <li>rawgreen-mean: a média sobre a região do valor G (cor verde) </li>
    <li>exred-mean: mede o excesso de vermelho: (2R - (G + B)) </li>
    <li>exblue-mean: mede o excesso de azul: (2B - (G + R)) </li>
    <li>exgreen-mean: mede o excesso de verde:  (2G - (R + B)) </li>
    <li>value-mean: transformação não-linear 3-d de RGB </li>
    <li>saturatoin-mean: média de saturação do RGB</li>
    <li>hue-mean: média de tonalidade do RGB </li>
    <b><li style="color: red">y-i: classe a ser inferida (ver figura acima)</li></b>
</ol>

<a href="https://storm.cis.fordham.edu/~gweiss/data-mining/weka-data/segment-challenge.arff">**Referência**</a>

**Atividade 7 - Leitura do Dataset e criação dos folds:** Leia o dataset [`segment.csv`](segment.csv). Faça a validação cruzada de 5 partições.

In [1]:
import pandas as pd
import hiplot as hip 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import optuna
from avaliacao import Experimento, OtimizacaoObjetivoRandomForest, OtimizacaoObjetivoArvoreDecisao
from resultado import Resultado,Fold
from metodo import ScikitLearnAprendizadoDeMaquina


df_dados = pd.read_csv('segment.csv')

#gera os folds
folds = Fold.gerar_k_folds(df_dados,val_k=5,col_classe="y-i",
                            num_repeticoes=1,seed=1,
                            num_folds_validacao=3,num_repeticoes_validacao=2)


# Fold: 0    Inicio: 0 -  Fim: 300 
# Fold: 0    Inicio: 0 -  Fim: 349 
# Fold: 1    Inicio: 349 -  Fim: 698 
# Fold: 2    Inicio: 698 -  Fim: 1047 
# Fold: 0    Inicio: 0 -  Fim: 349 
# Fold: 1    Inicio: 349 -  Fim: 698 
# Fold: 2    Inicio: 698 -  Fim: 1047 
# Fold: 1    Inicio: 300 -  Fim: 600 
# Fold: 0    Inicio: 0 -  Fim: 349 
# Fold: 1    Inicio: 349 -  Fim: 698 
# Fold: 2    Inicio: 698 -  Fim: 1047 
# Fold: 0    Inicio: 0 -  Fim: 349 
# Fold: 1    Inicio: 349 -  Fim: 698 
# Fold: 2    Inicio: 698 -  Fim: 1047 
# Fold: 2    Inicio: 600 -  Fim: 900 
# Fold: 0    Inicio: 0 -  Fim: 344 
# Fold: 1    Inicio: 344 -  Fim: 688 
# Fold: 2    Inicio: 688 -  Fim: 1034 
# Fold: 0    Inicio: 0 -  Fim: 344 
# Fold: 1    Inicio: 344 -  Fim: 688 
# Fold: 2    Inicio: 688 -  Fim: 1034 
# Fold: 3    Inicio: 900 -  Fim: 1200 
# Fold: 0    Inicio: 0 -  Fim: 352 
# Fold: 1    Inicio: 352 -  Fim: 704 
# Fold: 2    Inicio: 704 -  Fim: 1058 
# Fold: 0    Inicio: 0 -  Fim: 352 
# Fold: 1    Inicio: 3

**Atividade 8 - Variação de parametros e exibição e analise de resultados:** Aplique, pelo menos, os métodos RandomForest e Árvore de Decisão no problema variando os parametros (no mínimo, da mesma forma que foi variado na Parte 2). Apresente os resultados faça uma analise e responda, pelo menos: quais são as classes mais dificieis/fácieis de prever? Quais se confundem mais? Qual é o melhor método de classificação? Quais são os melhores parametros para cada método de aprendizado de máquina?

Para fazer a análise por classe, use as predições de todos os folds (apenas uma repetição) e gere a matriz de confusão. Qualquer dúvida, veja a aula sobre avaliação de métodos de aprendizado de máquina. A classe Resultado implementa essa matriz. 

In [2]:
# Experimento 1: Árvore de Decisão

clf_dtree1 = DecisionTreeClassifier(random_state=1)

ml_method1 = ScikitLearnAprendizadoDeMaquina(clf_dtree1)

exp1 = Experimento(folds, ml_method1, OtimizacaoObjetivoArvoreDecisao, num_trials=10,
                    sampler=optuna.samplers.TPESampler(seed=1, n_startup_trials=3))

exp1.calcula_resultados()


[I 2022-10-25 16:14:10,161] A new study created in memory with name: no-name-bd0c3da9-4d33-4f96-8fe1-98b4bac330f5
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:123: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
[I 2022-10-25 16:14:10,262] Trial 0 finished with value: 0.8560398296809577 and parameters: {'min_samples_split': 0.208511002351287}. Best is trial 0 with value: 0.8560398296809577.
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:123: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_spli

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:123: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
[I 2022-10-25 16:14:10,781] Trial 6 finished with value: 0.8980567478299003 and parameters: {'min_samples_split': 0.1689258317519885}. Best is trial 2 with value: 0.9296400558641417.
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:123: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
[I 2022-10-25 16:14:10,876] Trial 7 finished with value: 0.9257203572644469 and parameters: {'min_sam

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 0 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 4 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 6 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
[I 2022-10-25 16:14:11,428] Trial 3 finished with value: 0.48867495612996525

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:123: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
[I 2022-10-25 16:14:11,810] Trial 7 finished with value: 0.9333955018162436 and parameters: {'min_samples_split': 0.0015644497781362367}. Best is trial 7 with value: 0.9333955018162436.
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:123: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
[I 2022-10-25 16:14:11,901] Trial 8 finished with value: 0.8667099044703478 and parameters: {'min_

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 0 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 2 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 6 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: U

...

[I 2022-10-25 16:14:12,895] A new study created in memory with name: no-name-7ace5ff8-84a6-4c87-ae99-58f968dd6214
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:123: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 4 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 4 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos p

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:123: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
[I 2022-10-25 16:14:13,439] Trial 6 finished with value: 0.9290242197272739 and parameters: {'min_samples_split': 0.13159357805031466}. Best is trial 5 with value: 0.9450344971717682.
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:123: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: N

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 2 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 6 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 0 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: U

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:123: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
[I 2022-10-25 16:14:13,891] Trial 2 finished with value: 0.9358343406831842 and parameters: {'min_samples_split': 0.013693796598963082}. Best is trial 2 with value: 0.9358343406831842.
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:123: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
[I 2022-10-25 16:14:13,976] Trial 3 finished with value: 0.9355878053828487 and parameters: {'min_s

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:123: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
[I 2022-10-25 16:14:14,296] Trial 7 finished with value: 0.9332645679639663 and parameters: {'min_samples_split': 0.024085796450709723}. Best is trial 2 with value: 0.9358343406831842.
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:123: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: 

In [4]:
from collections import defaultdict

listRes1 = exp1.resultados

for i in range (0, len(listRes1)):
    # Anexando F1 por classe de cada fold
    d1 = listRes1[0].f1_por_classe
    d2 = listRes1[1].f1_por_classe
    d3 = listRes1[2].f1_por_classe
    d4 = listRes1[3].f1_por_classe
    d5 = listRes1[4].f1_por_classe
    
    dd = defaultdict(list)

    for d in (d1,d2,d3,d4,d5):
        for key, value in d.items():
            dd[key].append(value)

for key, value in dd.items():
    print(key, value)
print("\n")

# Soma dos valores anexados
dd2 = dict(zip(dd.keys(), [[sum(item)] for item in dd.values()]))

# Média F1 por classe
for key in dd2:
    dd2[key] = [sum(dd2[key])/len(listRes1)]

# Print em ordem decrescente
for k in sorted(dd2, key=dd2.get, reverse=True):
    print(k, dd2[k])

0 [0.9722222222222222, 0.9879518072289156, 0.9473684210526315, 0.9565217391304348, 0.9545454545454545]
1 [1.0, 1.0, 1.0, 0.989247311827957, 1.0]
2 [0.8857142857142858, 0.853932584269663, 0.9285714285714285, 0.776470588235294, 0.8351648351648352]
3 [0.9514563106796117, 0.8787878787878788, 0.9072164948453608, 0.8444444444444444, 0.8831168831168831]
4 [0.8717948717948718, 0.8282828282828283, 0.8947368421052632, 0.7246376811594202, 0.7727272727272727]
5 [1.0, 0.968421052631579, 0.967741935483871, 0.9876543209876543, 1.0]
6 [0.9846153846153847, 1.0, 1.0, 1.0, 1.0]


1 [0.9978494623655914]
6 [0.9969230769230769]
5 [0.9847634618206207]
0 [0.9637219288359317]
3 [0.8930044023748358]
2 [0.8559707443911012]
4 [0.8184358992139312]


**Quais classes são mais difíceis/fáceis de prever?**

Conforme a média de F1 por classe dos folds em ordem decrescente, temos:
- (2) sky
- (7) grass
- (6) path
- (1) brickface
- (4) cement
- (3) foliage
- (5) window

Portanto, as classes (2) e (7) demonstram F1 mais elevado, ou seja, seus valores de precisão de revocação são maiores que as demais e podemos afirmar que são mais fáceis de prever. Já as classes (3) e (5) são mais difíceis, conforme valores de F1 mais baixo.

In [5]:
for i in range (0, len(listRes1)):
    # Matriz de confusão para cada fold 
    # Classe Real X Predito como...
    for key, value in listRes1[i].mat_confusao.items():
        print(key, value)
    print("\n")
    
    # Macro F1 para cada fold
    #print(listRes[i].macro_f1)
    #print("\n")

0 {0: 35, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0}
1 {0: 0, 1: 53, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
2 {0: 1, 1: 0, 2: 31, 3: 0, 4: 4, 5: 0, 6: 0}
3 {0: 0, 1: 0, 2: 0, 3: 49, 4: 3, 5: 0, 6: 0}
4 {0: 0, 1: 0, 2: 1, 3: 2, 4: 34, 5: 0, 6: 0}
5 {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 53, 6: 0}
6 {0: 0, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 32}


0 {0: 41, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
1 {0: 0, 1: 47, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
2 {0: 0, 1: 0, 2: 38, 3: 1, 4: 6, 5: 0, 6: 0}
3 {0: 0, 1: 0, 2: 0, 3: 29, 4: 0, 5: 0, 6: 0}
4 {0: 1, 1: 0, 2: 4, 3: 6, 4: 41, 5: 0, 6: 0}
5 {0: 0, 1: 0, 2: 2, 3: 1, 4: 0, 5: 46, 6: 0}
6 {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 37}


0 {0: 36, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0}
1 {0: 0, 1: 35, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
2 {0: 0, 1: 0, 2: 39, 3: 0, 4: 1, 5: 0, 6: 0}
3 {0: 2, 1: 0, 2: 1, 3: 44, 4: 3, 5: 3, 6: 0}
4 {0: 1, 1: 0, 2: 3, 3: 0, 4: 34, 5: 0, 6: 0}
5 {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 45, 6: 0}
6 {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 52}


0 {0: 44, 1: 0, 2: 2, 3: 1, 

**Quais se confundem mais?**

Pelos resultados das matrizes de confusão dos folds, nota-se que as classes (3), (4) e (5) apresentam mais erros em suas predições, uma vez que elas possuem contadores em outras classes. O que condiz com os resultados de F1 obtidos anteriormente.

In [18]:
# Experimento 2: RandomForest

clf_dtree2 = RandomForestClassifier(random_state=1)

ml_method2 = ScikitLearnAprendizadoDeMaquina(clf_dtree2)

exp2 = Experimento(folds, ml_method2, OtimizacaoObjetivoRandomForest, num_trials=10,
                    sampler=optuna.samplers.TPESampler(seed=1, n_startup_trials=3))

exp2.calcula_resultados()

[I 2022-10-25 17:02:03,532] A new study created in memory with name: no-name-9c78bda5-61eb-4a2b-b482-d942f38c4584
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:141: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:142: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  max_features = trial.suggest_uniform('max_features', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the sh

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:141: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:142: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  max_features = trial.suggest_uniform('max_features', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 3 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 4 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há el

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 0 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 4 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há el

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:141: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:142: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  max_features = trial.suggest_uniform('max_features', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas

...

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 0 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 2 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 3 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: Data

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 0 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 2 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há el

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 0 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 2 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vect

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 2 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array 

In [16]:
from collections import defaultdict

listRes2 = exp2.resultados

# Acurácia por fold
for i in range (0, len(listRes1)):
    print(i, '->', listRes1[i].acuracia) # Árvore de Decisão
    print(i, '->', listRes2[i].acuracia) # RandomForest
    if listRes1[i].acuracia > listRes2[i].acuracia:
        print('Árvore de Decisão superior')
    else:
        print('RandomForest superior')
    print(listRes1[i].acuracia - listRes2[i].acuracia)

0 -> 0.9566666666666667
0 -> 0.96
RandomForest superior
-0.0033333333333332993
1 -> 0.93
1 -> 0.9066666666666666
Árvore de Decisão superior
0.023333333333333428
2 -> 0.95
2 -> 0.9133333333333333
Árvore de Decisão superior
0.036666666666666625
3 -> 0.9033333333333333
3 -> 0.9233333333333333
RandomForest superior
-0.020000000000000018
4 -> 0.92
4 -> 0.9266666666666666
RandomForest superior
-0.006666666666666599


**Qual é o melhor método de classificação?**

Seguindo o resultado da acurácia dos dois métodos para este experimento, percebe-se que o método Árvore de Decisão vence em 3 dos 5 folds com uma diferença significativa e nos outros 2 que perde com uma diferença menor. Portanto, existe uma tendência estatística para afirmar que, para este problema, o método Árvore de Decisão gera um modelo de aprendizado de máquina mais preciso. Vale ressaltar que os parâmetros de entrada influenciam neste resultado, então sem um estudo mais profundo sobre a comparação de métodos, não é possível afirmar com exatidão qual é melhor.

In [13]:
# Árvore de Decisão
studdy_fold_0 = exp1.studies_per_fold[0]
studdy_fold_0.trials_dataframe().sort_values("value",ascending=False)

,number,value,datetime_start,datetime_complete,duration,params_min_samples_split,state
2,2,0.929640,2022-10-25 16:14:10.344430,2022-10-25 16:14:10.432880,0 days 00:00:00.088450,0.000057,COMPLETE
9,9,0.927782,2022-10-25 16:14:10.956893,2022-10-25 16:14:11.040163,0 days 00:00:00.083270,0.120755,COMPLETE
3,3,0.927510,2022-10-25 16:14:10.433921,2022-10-25 16:14:10.528295,0 days 00:00:00.094374,0.005470,COMPLETE
7,7,0.925720,2022-10-25 16:14:10.782769,2022-10-25 16:14:10.875810,0 days 00:00:00.093041,0.102073,COMPLETE
4,4,0.924023,2022-10-25 16:14:10.529586,2022-10-25 16:14:10.620314,0 days 00:00:00.090728,0.041847,COMPLETE
6,6,0.898057,2022-10-25 16:14:10.695853,2022-10-25 16:14:10.781537,0 days 00:00:00.085684,0.168926,COMPLETE
0,0,0.856040,2022-10-25 16:14:10.163171,2022-10-25 16:14:10.262166,0 days 00:00:00.098995,0.208511,COMPLETE
1,1,0.665465,2022-10-25 16:14:10.263341,2022-10-25 16:14:10.342737,0 days 00:00:00.079396,0.360162,COMPLETE
8,8,0.665465,2022-10-25 16:14:10.877021,2022-10-25 16:14:10.955866,0 days 00:00:00.078845,0.308646,COMPLETE
5,5,0.519579,2022-10-25 16:14:10.621438,2022-10-25 16:14:10.694066,0 days 00:00:00.072628,0.483598,COMPLETE


In [19]:
# RandomForest
studdy_fold_0 = exp2.studies_per_fold[0]
studdy_fold_0.trials_dataframe().sort_values("value",ascending=False)

,number,value,datetime_start,datetime_complete,duration,params_max_features,params_min_samples_split,params_num_arvores,state
5,5,0.948209,2022-10-25 17:02:04.042271,2022-10-25 17:02:04.236840,0 days 00:00:00.194569,0.496695,0.012072,5,COMPLETE
8,8,0.932101,2022-10-25 17:02:04.520289,2022-10-25 17:02:04.702251,0 days 00:00:00.181962,0.338017,0.002669,4,COMPLETE
3,3,0.928594,2022-10-25 17:02:03.766097,2022-10-25 17:02:03.904260,0 days 00:00:00.138163,0.146324,0.006228,5,COMPLETE
9,9,0.894779,2022-10-25 17:02:04.703776,2022-10-25 17:02:04.828645,0 days 00:00:00.124869,0.333432,0.108441,3,COMPLETE
2,2,0.876788,2022-10-25 17:02:03.678738,2022-10-25 17:02:03.764225,0 days 00:00:00.085487,0.172780,0.093130,2,COMPLETE
1,1,0.745871,2022-10-25 17:02:03.614268,2022-10-25 17:02:03.677492,0 days 00:00:00.063224,0.073378,0.151166,1,COMPLETE
7,7,0.660059,2022-10-25 17:02:04.391776,2022-10-25 17:02:04.519024,0 days 00:00:00.127248,0.495729,0.313319,3,COMPLETE
6,6,0.640098,2022-10-25 17:02:04.238156,2022-10-25 17:02:04.390194,0 days 00:00:00.152038,0.498236,0.371370,4,COMPLETE
0,0,0.615022,2022-10-25 17:02:03.533896,2022-10-25 17:02:03.612421,0 days 00:00:00.078525,0.360162,0.208511,1,COMPLETE
4,4,0.534180,2022-10-25 17:02:03.905963,2022-10-25 17:02:04.040773,0 days 00:00:00.134810,0.007422,0.493828,5,COMPLETE


**Quais são os melhores parametros para cada método de aprendizado de máquina?**

- Árvore de Decisão:

Conforme a tabela gerada pelo framework Optuna, o melhor valor de `params_min_samples_split` encontrado para otimizar o ganho de macro F1 dentro do fold 0 foi: `0.000057`. Nota-se que este é a porcentagem mais baixa dentre os outros trials, mas não é plausível afirmar que sempre será a menor, porque o experimento foi feito para solucionar este problema em específico, além disso, o resultado pode variar conforme usamos outros parâmetros de entrada.

- RandomForest

Observando a segunda tabela gerada pelo framework, o resultado que provocou um macro F1 mais elevado foi a seguinte combinação de parâmetros: `params_max_features = 0.496695`, `params_min_samples_split = 0.012072` e `params_num_arvores = 5`. 

Olhando para as diferenças de valor entre os macro F1 de cada trial, nos dois métodos separadamente, conclui-se que para o RandomForest há um intervalo maior entre estes valores e a Árvore de Decisão possui diferenças que chegam a terceira casa decimal. Isto é um indicativo de que essa diferença se deve ao fato do método RandomForest necessitar de mais parâmetros, portanto, mais possíveis combinações. Aumentando o número de trials, essa diferença diminuirá, veja:

In [22]:
# Experimento 2: RandomForest - mais trials

exp2 = Experimento(folds, ml_method2, OtimizacaoObjetivoRandomForest, num_trials=20,
                    sampler=optuna.samplers.TPESampler(seed=1, n_startup_trials=3))

exp2.calcula_resultados()

studdy_fold_0 = exp2.studies_per_fold[0]
studdy_fold_0.trials_dataframe().sort_values("value",ascending=False)

[I 2022-10-25 17:04:07,698] A new study created in memory with name: no-name-07c74a44-e6ad-4594-b696-0c70e71cf972
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:141: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:142: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  max_features = trial.suggest_uniform('max_features', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the sh

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:141: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:142: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  max_features = trial.suggest_uniform('max_features', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 3 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 4 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há el

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 0 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há elementos previstos para a classe 4 precisão foi definida como zero.
  warnings.warn("Não há elementos previstos para a classe "+str(classe)+" precisão foi definida como zero.", UndefinedMetricWarning)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/resultado.py:78: UndefinedMetricWarning: Não há el

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:141: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:142: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  max_features = trial.suggest_uniform('max_features', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas

...

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:141: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  min_samples = trial.suggest_uniform('min_samples_split', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/avaliacao.py:142: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  max_features = trial.suggest_uniform('max_features', 0, 0.5)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = self.ml_method.fit(x_treino, y_treino)
/home/thomas/PycharmProjects/avaliacaoParte2_Paulo_Thomas/metodo.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

,number,value,datetime_start,datetime_complete,duration,params_max_features,params_min_samples_split,params_num_arvores,state
12,12,0.949000,2022-10-25 17:04:09.308099,2022-10-25 17:04:09.499897,0 days 00:00:00.191798,0.401876,0.011416,5,COMPLETE
5,5,0.948209,2022-10-25 17:04:08.244107,2022-10-25 17:04:08.421780,0 days 00:00:00.177673,0.496695,0.012072,5,COMPLETE
8,8,0.932101,2022-10-25 17:04:08.686982,2022-10-25 17:04:08.841602,0 days 00:00:00.154620,0.338017,0.002669,4,COMPLETE
3,3,0.928594,2022-10-25 17:04:07.967821,2022-10-25 17:04:08.103131,0 days 00:00:00.135310,0.146324,0.006228,5,COMPLETE
11,11,0.925695,2022-10-25 17:04:09.157744,2022-10-25 17:04:09.306874,0 days 00:00:00.149130,0.283282,0.032254,4,COMPLETE
16,16,0.924852,2022-10-25 17:04:10.013278,2022-10-25 17:04:10.173176,0 days 00:00:00.159898,0.442665,0.060276,4,COMPLETE
19,19,0.924834,2022-10-25 17:04:10.407621,2022-10-25 17:04:10.582952,0 days 00:00:00.175331,0.465529,0.045652,5,COMPLETE
13,13,0.907780,2022-10-25 17:04:09.501317,2022-10-25 17:04:09.681200,0 days 00:00:00.179883,0.432126,0.081398,5,COMPLETE
9,9,0.894779,2022-10-25 17:04:08.843042,2022-10-25 17:04:08.989147,0 days 00:00:00.146105,0.333432,0.108441,3,COMPLETE
2,2,0.876788,2022-10-25 17:04:07.871928,2022-10-25 17:04:07.965641,0 days 00:00:00.093713,0.172780,0.093130,2,COMPLETE
